<a href="https://colab.research.google.com/github/brendafpacheco/Projeto-01/blob/master/Projeto_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# abrir a conexão com o banco de dados
import sqlite3
import pandas as pd
conn = sqlite3.connect("database.db")

In [ ]:
# consulta dos dados do banco de dados
consulta_atividade = "SELECT * FROM flight_loyalty_history flh INNER JOIN flight_activity fa ON (flh.loyalty_number = fa.loyalty_number)"
df_atividade = pd.read_sql_query( consulta_atividade, conn )

In [ ]:
# Número de dados faltantes em cada coluna
df_atividade.isna().sum() #coloca false nas colunas que têm dados e true nas colunas que são nulas

# Selecionar somente as colunas que contém números
colunas = ["year", "month","flights_booked", "flights_with_companions", "total_flights", "distance", "points_accumulated", "salary", "clv", "loyalty_card"]
df_colunas_numericas = df_atividade.loc[:, colunas]

# Remover as linhas que contém dados faltantes
df_dados_completos = df_colunas_numericas.dropna()

# Verificar se existe dados faltantes
df_dados_completos.isna().sum()


In [11]:
#Machine Learning

from sklearn import tree as tr

x_atributos = df_dados_completos.drop( columns="loyalty_card" )
y_rotulos = df_dados_completos.loc[:, "loyalty_card"]

#definicao do algoritmo
modelo = tr.DecisionTreeClassifier(max_depth=2)

#treinamento do algoritmo
modelo_treinado = modelo.fit( x_atributos, y_rotulos )

In [12]:
!pip install gradio
import gradio as gr
import numpy as np

In [ ]:
def predict(*args):
  x_novo = np.array( [args] ).reshape( 1, -1 )
  labels = modelo_treinado.predict_proba( x_novo )
  return {"Aurora": labels[0][0], "Nova": labels[0][1], "Star": labels[0][2]}

with gr.Blocks() as demo:
  gr.Markdown(""" # Propensão de compra """)

  with gr.Row():

    with gr.Column():
      gr.Markdown(""" # Atributos do cliente """)
      year = gr.Slider( label="year", minimum=2017, maximum=2018, step=1, randomize=True)
      month = gr.Slider( label="month", minimum=1, maximum=12, step=1, randomize=True)
      flights_booked = gr.Slider( label="flights_booked", minimum=0, maximum=21, step=1, randomize=True)
      flights_with_companions = gr.Slider( label="flights_with_companions", minimum=0, maximum=11, step=1, randomize=True)
      total_flights = gr.Slider( label="total_flights", minimum=0, maximum=32, step=1, randomize=True)
      distance = gr.Slider( label="distance", minimum=0, maximum=6293, step=1, randomize=True)
      points_accumulated = gr.Slider( label="points_accumulated", minimum=0.00, maximum=676.5, step=1, randomize=True)
      salary = gr.Slider( label="salary", minimum=58486.00, maximum=407228.0, step=1, randomize=True)
      clv = gr.Slider( label="clv", minimum=2017, maximum=83325.38, step=1, randomize=True)

      with gr.Row():
          with gr.Row():
            gr.Markdown(""" # Botão de previsão """)
            predict_btn = gr.Button( value="Previsão" )

    with gr.Column():
      gr.Markdown(""" # Propensão de compra do cliente """)
      label = gr.Label()

  # Botão predict
  predict_btn.click(
      fn=predict,
      inputs=[
        year,
        month,
        flights_booked,
        flights_with_companions,
        total_flights,
        distance,
        points_accumulated,
        salary,
        clv
      ],
      outputs=[label])

demo.launch(share=True, debug=True)